In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import Resize, ToTensor, Normalize, RandomRotation, RandomHorizontalFlip
from sklearn.metrics import classification_report
from torchvision import transforms 

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = FashionMNIST(root='./data', train=True, download=True,
                             transform=transforms.Compose([
                                 Resize((64, 64)),
                                 RandomRotation(15),  # Random rotation up to 15 degrees
                                 RandomHorizontalFlip(),  # Random horizontal flip
                                 ToTensor(),
                                 Normalize((0.5,), (0.5,))
                             ]))

test_dataset = FashionMNIST(root='./data', train=False, download=True,
                            transform=transforms.Compose([
                                Resize((64, 64)),
                                ToTensor(),
                                Normalize((0.5,), (0.5,))
                            ]))

100%|██████████| 26421880/26421880 [00:01<00:00, 16461474.73it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 295578.75it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5524677.33it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11275340.47it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [4]:
class ImprovedANN(nn.Module):
    def __init__(self):
        super(ImprovedANN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 64, 2048)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(2048, 1024)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(512, 256)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(256, 128)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(128, 64)
        self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        return x


In [5]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    return train_loss, train_accuracy

def evaluate(model, criterion, data_loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    predictions = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            predictions.extend(predicted.tolist())

    loss = running_loss / len(data_loader)
    accuracy = correct / total
    return loss, accuracy, predictions


In [6]:
improved_model = ImprovedANN()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(improved_model.parameters(), lr=0.0001)

In [9]:
for epoch in range(85):  # Train for 10 epochs
    train_loss, train_accuracy = train(improved_model, criterion, optimizer, train_loader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

Epoch 1: Train Loss: 0.3032, Train Accuracy: 0.8861
Epoch 2: Train Loss: 0.2971, Train Accuracy: 0.8895
Epoch 3: Train Loss: 0.2887, Train Accuracy: 0.8902
Epoch 4: Train Loss: 0.2861, Train Accuracy: 0.8923
Epoch 5: Train Loss: 0.2797, Train Accuracy: 0.8947
Epoch 6: Train Loss: 0.2722, Train Accuracy: 0.8972
Epoch 7: Train Loss: 0.2702, Train Accuracy: 0.8977
Epoch 8: Train Loss: 0.2615, Train Accuracy: 0.9013
Epoch 9: Train Loss: 0.2600, Train Accuracy: 0.9026
Epoch 10: Train Loss: 0.2543, Train Accuracy: 0.9050
Epoch 11: Train Loss: 0.2487, Train Accuracy: 0.9057
Epoch 12: Train Loss: 0.2455, Train Accuracy: 0.9070
Epoch 13: Train Loss: 0.2393, Train Accuracy: 0.9096
Epoch 14: Train Loss: 0.2371, Train Accuracy: 0.9105
Epoch 15: Train Loss: 0.2319, Train Accuracy: 0.9112
Epoch 16: Train Loss: 0.2264, Train Accuracy: 0.9152
Epoch 17: Train Loss: 0.2222, Train Accuracy: 0.9156
Epoch 18: Train Loss: 0.2171, Train Accuracy: 0.9168
Epoch 19: Train Loss: 0.2153, Train Accuracy: 0.9176
Ep

In [10]:
test_loss, test_accuracy, predictions = evaluate(improved_model, criterion, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Step 9: Print the classification report
true_labels = [label for _, label in test_dataset]
print(classification_report(true_labels, predictions))

Test Loss: 0.6762, Test Accuracy: 0.8875
              precision    recall  f1-score   support

           0       0.83      0.82      0.83      1000
           1       0.98      0.98      0.98      1000
           2       0.82      0.82      0.82      1000
           3       0.89      0.90      0.89      1000
           4       0.82      0.79      0.81      1000
           5       0.98      0.96      0.97      1000
           6       0.70      0.72      0.71      1000
           7       0.94      0.95      0.95      1000
           8       0.96      0.97      0.97      1000
           9       0.96      0.96      0.96      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

